In [53]:
import yt_dlp
import os
from moviepy.video.io.VideoFileClip import VideoFileClip
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import json

In [54]:
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [55]:
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(f"Using {torch_dtype} dtype")

Using torch.float32 dtype


In [56]:
model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

In [57]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use mps


In [58]:
def download_youtube_video_yt_dlp(url):
    # extract video ID from the URL
    video_id = url.split("v=")[-1]
    if "&" in video_id:
        video_id = video_id.split("&")[0]

    ydl_opts = {
        "outtmpl": f"{video_id}.%(ext)s",  # Output path and filename
        "format": "best",  # Select the best single file (video + audio)
        "merge_output_format": None,  # Avoid merging, stick to single stream
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print("Download completed successfully!")
        return video_id
    except Exception as e:
        print(f"An error occurred: {e}")

In [59]:
def get_video_chunk_names(path):
    # extract video ID from the filename
    video_id = path.split('__')[0]
    chunk_name = path.split('__')[2].split('.')[0]
    return video_id, chunk_name

In [60]:
def split_audio_video(video_file):
    try:
        # extract video ID from the filename
        video_id, chunk_name = get_video_chunk_names(video_file)
        print(f"Video ID: {video_id}, Chunk Name: {chunk_name}")

        # import video
        video_chunk = VideoFileClip(video_file)

        # Split audio and video
        audio_path = os.path.join(f"{video_id}__audio__{chunk_name}.mp3")
        video_path = os.path.join(f"{video_id}__video__{chunk_name}.mp4")
        print(f"Audio path: {audio_path}, Video path: {video_path}")

        # Write audio to file
        video_chunk.audio.write_audiofile(audio_path)

        # Write video to file
        video_only = video_chunk.without_audio()
        video_only.write_videofile(f"{os.getcwd()}/data/videos/{video_path}", codec="libx264", audio_codec="aac")

        # Close the video clip
        video_chunk.close()
        video_only.close()

        # delete chunk video file
        os.remove(video_file)

        print("Audio and video split successfully!")
        return audio_path
    except Exception as e:
        print(f"An error occurred: {e}")

In [61]:
def clip_video_into_chunks(video_id, chunk_length=10):
    input_file = video_id + '.mp4'
    try:
        video = VideoFileClip(input_file)
        duration = int(video.duration)
        chunks = []

        for start in range(0, duration, chunk_length):
            end = min(start + chunk_length, duration)
            chunk = video.subclipped(start, end)
            chunk_filename = f"{video_id}__chunk__{start // chunk_length}.mp4"
            chunk.write_videofile(chunk_filename, codec="libx264")
            chunks.append(chunk_filename)

        video.close()

        # detele original video file
        os.remove(input_file)

        return chunks
    except Exception as e:
        print(f"An error occurred while splitting the video: {e}")
        return []

In [62]:
def audio_to_text(audio_file):
    result = pipe(audio_file)
    text = result["text"]
    # extract video ID from the filename
    video_id, chunk_name = get_video_chunk_names(audio_file)
    # save text to JSON file
    data = {
        "video_id": video_id,
        "chunk_name": chunk_name,
        "text": text
    }

    # Save the data to a JSON file
    json_file = f"{video_id}__text__{chunk_name}.json"
    with open(f"{os.getcwd()}/data/labels/{json_file}", "w") as f:
        json.dump(data, f, indent=4)

    print(f"Text saved to {json_file}")

    # remove audio file
    os.remove(audio_file)

In [63]:
# get list of YT video urls
videos = ['https://www.youtube.com/watch?v=KjB6r-HDDI0']

In [64]:
for url in videos:
    video_name = download_youtube_video_yt_dlp(url)
    video_chunks = clip_video_into_chunks(video_name, chunk_length=10)
    for chunk in video_chunks:
        audio_path = split_audio_video(chunk)
        audio_to_text(audio_path)

[youtube] Extracting URL: https://www.youtube.com/watch?v=KjB6r-HDDI0
[youtube] KjB6r-HDDI0: Downloading webpage
[youtube] KjB6r-HDDI0: Downloading tv client config
[youtube] KjB6r-HDDI0: Downloading player 22f02d3d-main
[youtube] KjB6r-HDDI0: Downloading tv player API JSON
[youtube] KjB6r-HDDI0: Downloading ios player API JSON
[youtube] KjB6r-HDDI0: Downloading m3u8 information
[info] KjB6r-HDDI0: Downloading 1 format(s): 18
[download] Destination: KjB6r-HDDI0.mp4
[download] 100% of    4.93MiB in 00:00:00 at 29.15MiB/s    
Download completed successfully!
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'encoder': 'Google'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 316, 'fps': 23.976023976023978, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Me

MoviePy - Done.
MoviePy - Writing video KjB6r-HDDI0__chunk__0.mp4



MoviePy - Done !
MoviePy - video ready KjB6r-HDDI0__chunk__0.mp4
MoviePy - Building video KjB6r-HDDI0__chunk__1.mp4.
MoviePy - Writing audio in KjB6r-HDDI0__chunk__1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video KjB6r-HDDI0__chunk__1.mp4



MoviePy - Done !
MoviePy - video ready KjB6r-HDDI0__chunk__1.mp4
MoviePy - Building video KjB6r-HDDI0__chunk__2.mp4.
MoviePy - Writing audio in KjB6r-HDDI0__chunk__2TEMP_MPY_wvf_snd.mp3


...

frame_index:  85%|████████▍ | 203/240 [00:00<00:00, 1009.44it/s, now=None]/Users/emmettstorts/.local/share/virtualenvs/slip-ml-D1sQfDoh/lib/python3.11/site-packages/moviepy/video/io/ffmpeg_reader.py:178: UserWarning: In file KjB6r-HDDI0__chunk__8.mp4, 691200 bytes wanted but 0 bytes read at frame index 239 (out of a total 240 frames), at time 9.97/10.01 sec. Using the last valid frame instead.
  warnings.warn(
/Users/emmettstorts/.local/share/virtualenvs/slip-ml-D1sQfDoh/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


MoviePy - Done !
MoviePy - video ready /Users/emmettstorts/Documents/slip-ml/data/videos/KjB6r-HDDI0__video__8.mp4
Audio and video split successfully!
Text saved to KjB6r-HDDI0__text__8.json
Video ID: KjB6r-HDDI0, Chunk Name: 9
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 133, 'fps': 23.976023976023978, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 

MoviePy - Done.
MoviePy - Building video /Users/emmettstorts/Documents/slip-ml/data/videos/KjB6r-HDDI0__video__9.mp4.
MoviePy - Writing video /Users/emmettstorts/Documents/slip-ml/data/videos/KjB6r-HDDI0__video__9.mp4



frame_index:  99%|█████████▊| 213/216 [00:00<00:00, 1088.91it/s, now=None]/Users/emmettstorts/.local/share/virtualenvs/slip-ml-D1sQfDoh/lib/python3.11/site-packages/moviepy/video/io/ffmpeg_reader.py:178: UserWarning: In file KjB6r-HDDI0__chunk__9.mp4, 691200 bytes wanted but 0 bytes read at frame index 215 (out of a total 216 frames), at time 8.97/9.01 sec. Using the last valid frame instead.
  warnings.warn(
/Users/emmettstorts/.local/share/virtualenvs/slip-ml-D1sQfDoh/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


MoviePy - Done !
MoviePy - video ready /Users/emmettstorts/Documents/slip-ml/data/videos/KjB6r-HDDI0__video__9.mp4
Audio and video split successfully!
Text saved to KjB6r-HDDI0__text__9.json
